In [2]:
# Load model directly
from transformers import AutoImageProcessor, VideoMAEForVideoClassification, VideoMAEFeatureExtractor, VideoMAEModel, VideoMAEForPreTraining

processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-large-finetuned-kinetics")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-large")
classifier = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-large-finetuned-kinetics")

In [9]:
import os 
from torchvision import transforms
import matplotlib.pyplot as plt

from PIL import Image
dir = '/Users/tannicholas/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/01December_2011_Thursday_tagesschau-3479'
# PyTorch transformation to resize and convert to tensor
image_paths = os.listdir(dir)
images = [] 
for path in image_paths: 
    actual_path = os.path.join(dir, path)
    
    # Load the image
    img = Image.open(actual_path).convert("RGB")  # Ensures image is in RGB format
    images.append(img)

processed_images = processor(images=[images[0:16],images[8:24], images[16:32]] , return_tensors="pt")
print(processed_images.pixel_values.shape)

print(images[0:16])

torch.Size([3, 16, 3, 224, 224])
[<PIL.Image.Image image mode=RGB size=210x260 at 0x2A95361D0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA2E3C10>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AFF40F90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA366C90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AD4017D0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA340F50>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA364310>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2A96BBC10>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2A9DDAFD0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2A9DDAF10>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA690D10>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2A9E0E950>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AA6B8B90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2A9D9B510>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2AD400190>, <PIL.Image.Image image mode=RGB size

In [3]:
import os 
from torchvision import transforms
import matplotlib.pyplot as plt
import math 
from PIL import Image
dir = '/Users/tannicholas/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/01December_2011_Thursday_tagesschau-3479'
# PyTorch transformation to resize and convert to tensor
image_paths = os.listdir(dir)
img_sample = [] 
for path in image_paths: 
    actual_path = os.path.join(dir, path)
    
    # Load the image
    img = Image.open(actual_path).convert("RGB")  # Ensures image is in RGB format
    img_sample .append(img)
num_clips = math.ceil((len(img_sample)-8) / 8)
num_frames = num_clips * 8 +8
# if not enough frames, repeat the last frame
img_sample.extend([img_sample[-1]]*(num_frames-len(img_sample)))

clip_sample = [img_sample[i:i+16] for i in range(0, len(img_sample), 8) 
               if len(img_sample[i:i+16])==16 if len(img_sample)-i>8]
print(clip_sample)
print(len(clip_sample[3]))
processed_images = processor(images=clip_sample , return_tensors="pt")
print(processed_images.pixel_values.shape)

[[<PIL.Image.Image image mode=RGB size=210x260 at 0x2CF8E6F90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x280AE50D0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x282D97010>, <PIL.Image.Image image mode=RGB size=210x260 at 0x283CE94D0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x28279A950>, <PIL.Image.Image image mode=RGB size=210x260 at 0x282DE8B10>, <PIL.Image.Image image mode=RGB size=210x260 at 0x282E6EED0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x282E6FF50>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8F010>, <PIL.Image.Image image mode=RGB size=210x260 at 0x282E6FF90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8E390>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8E2D0>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8E150>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8DF90>, <PIL.Image.Image image mode=RGB size=210x260 at 0x1067B5A50>, <PIL.Image.Image image mode=RGB size=210x260 at 0x2D6C8DC90>], [<PIL

/opt/homebrew/lib/python3.11/site-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  return torch.tensor(value)


In [4]:
output = model(processed_images.pixel_values, output_hidden_states=True)
print(output.keys())
print(output.last_hidden_state.shape)


odict_keys(['last_hidden_state', 'hidden_states'])
torch.Size([4, 1568, 1024])


In [72]:
model

VideoMAEModel(
  (embeddings): VideoMAEEmbeddings(
    (patch_embeddings): VideoMAEPatchEmbeddings(
      (projection): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
    )
  )
  (encoder): VideoMAEEncoder(
    (layer): ModuleList(
      (0-23): 24 x VideoMAELayer(
        (attention): VideoMAESdpaAttention(
          (attention): VideoMAESdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=False)
            (key): Linear(in_features=1024, out_features=1024, bias=False)
            (value): Linear(in_features=1024, out_features=1024, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): VideoMAESelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): VideoMAEIntermediate(
          (dense): Linear(in_features=1024, out_features=4096, bias=True)
          (intermedia

In [55]:
classifier

VideoMAEForVideoClassification(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0-23): 24 x VideoMAELayer(
          (attention): VideoMAESdpaAttention(
            (attention): VideoMAESdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
       

In [84]:
output= classifier(processed_images.pixel_values)
logits = output.logits
predicted_label = logits.argmax(-1).item()
print(classifier.config.id2label[predicted_label])

sign language interpreting


In [85]:
logits.shape

torch.Size([1, 400])

In [5]:
import torch 
with torch.no_grad():
    # Forward pass up to the encoder and normalization layer
    features = classifier.videomae(processed_images.pixel_values).last_hidden_state  # Shape: [batch_size, num_patches, feature_dim]
    print(f"features before layer norm: {features.shape}")
    # Apply the final LayerNorm before the classifier
    features1 = classifier.fc_norm(features.mean(dim=1))  # Shape: [batch_size, 1024]
    ## This is wrong features2 = classifier.fc_norm(features).mean(dim=1)  # Shape: [batch_size, 1024]
    label = classifier.classifier(features1)  # Shape: [batch_size, num_labels]
print("Encoded video features shape:", features1.shape)
print(label.shape)
print(label.argmax(-1))

features before layer norm: torch.Size([4, 1568, 1024])
Encoded video features shape: torch.Size([4, 1024])
torch.Size([4, 400])
tensor([303, 303, 303, 303])


In [11]:
features1.shape

torch.Size([4, 1024])

In [21]:
tensor = features1.repeat(2,1)
tensor.shape

torch.Size([8, 1024])

In [26]:
tensor = torch.tile(features1, (2,1))
tensor.shape

torch.Size([8, 1024])

In [33]:
# Original tensor


# Add a new dimension and repeat along that dimension
expanded = features1.unsqueeze(1).repeat(1, 2, 1)
result = expanded.view(-1, features1.size(1))
print(result.shape)
result[0]
# Output:
# tensor([[1, 2],
#         [1, 2],
#         [2, 4],
#         [2, 4]])

torch.Size([8, 1024])


tensor([-0.0981,  0.1119, -0.0841,  ...,  0.1306, -0.2328,  0.0359])

In [32]:
result[1]

tensor([-0.0981,  0.1119, -0.0841,  ...,  0.1306, -0.2328,  0.0359])

In [29]:
tensor[1]

tensor([-0.0745,  0.2571, -0.1593,  ...,  0.0703, -0.2853,  0.0805])

In [36]:
torch.tensor([1,2,3,4])+ torch.tensor([1,2,3,4])



tensor([2, 4, 6, 8])